# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Castro,10,BR,1603409030,86,-24.79,-50.01,61.56,10.54
1,Rio Pardo de Minas,99,BR,1603409049,83,-15.61,-42.54,68.88,6.80
2,Qaanaaq,100,GL,1603408772,94,77.48,-69.36,20.35,8.52
3,Yar-Sale,91,RU,1603409050,96,66.83,70.83,15.78,8.50
4,Hong Kong,100,HK,1603408873,57,22.29,114.16,73.00,1.01
...,...,...,...,...,...,...,...,...,...
553,Masallātah,0,LY,1603409106,64,32.62,14.00,64.24,13.42
554,Konëvo,100,RU,1603409107,97,62.12,39.32,32.52,11.79
555,Kirkenær,100,NO,1603409107,99,60.46,12.06,41.00,1.99
556,Sangīn,0,AF,1603409107,43,32.07,64.84,46.29,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
49,Eilat,0,IL,1603409055,69,29.56,34.95,78.80,1.12
50,Gunnedah,0,AU,1603409055,64,-30.98,150.25,71.60,5.82
154,Morondava,0,MG,1603409065,86,-20.28,44.28,76.06,9.37
198,Abnūb,0,EG,1603409054,44,27.27,31.15,77.00,8.05
263,Doha,0,QA,1603409076,69,25.29,51.53,75.20,5.48
293,Bushehr,0,IR,1603409079,50,28.97,50.84,75.20,3.69
369,Sur,0,OM,1603409028,60,22.57,59.53,72.91,6.26
383,Digri,0,PK,1603409088,23,25.16,69.11,77.61,8.63
412,Beirut,0,LB,1603409091,60,33.89,35.49,73.40,2.24
443,Osmaniye,0,TR,1603409094,79,37.07,36.25,71.01,4.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,Eilat,IL,29.56,34.95,
50,Gunnedah,AU,-30.98,150.25,
154,Morondava,MG,-20.28,44.28,
198,Abnūb,EG,27.27,31.15,
263,Doha,QA,25.29,51.53,
293,Bushehr,IR,28.97,50.84,
369,Sur,OM,22.57,59.53,
383,Digri,PK,25.16,69.11,
412,Beirut,LB,33.89,35.49,
443,Osmaniye,TR,37.07,36.25,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 49: Eilat.
Closest hotel in Eilat is Al Manara, a Luxury Collection Hotel, Saraya Aqaba.
------------
Retrieving Results for Index 50: Gunnedah.
Closest hotel in Gunnedah is Gunnedah Hotel + VITA Restaurant.
------------
Retrieving Results for Index 154: Morondava.
Closest hotel in Morondava is Palissandre Cote Ouest resort & SPA.
------------
Retrieving Results for Index 198: Abnūb.
Missing field/result... skipping.
------------
Retrieving Results for Index 263: Doha.
Closest hotel in Doha is Al Najada Doha Hotel by Tivoli.
------------
Retrieving Results for Index 293: Bushehr.
Closest hotel in Bushehr is Delvar Hotel.
------------
Retrieving Results for Index 369: Sur.
Closest hotel in Sur is Sur Plaza Hotel.
------------
Retrieving Results for Index 383: Digri.
Closest hotel in Digri is Sindh Green Hotel.
------------
Retrieving Results for Index 412: Beirut.
Closest hotel in Beirut is Kempinski Summerland Hotel & Resort.
------------
Retrieving Results

In [10]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
49,Eilat,IL,29.56,34.95,"Al Manara, a Luxury Collection Hotel, Saraya A..."
50,Gunnedah,AU,-30.98,150.25,Gunnedah Hotel + VITA Restaurant
154,Morondava,MG,-20.28,44.28,Palissandre Cote Ouest resort & SPA
198,Abnūb,EG,27.27,31.15,
263,Doha,QA,25.29,51.53,Al Najada Doha Hotel by Tivoli
293,Bushehr,IR,28.97,50.84,Delvar Hotel
369,Sur,OM,22.57,59.53,Sur Plaza Hotel
383,Digri,PK,25.16,69.11,Sindh Green Hotel
412,Beirut,LB,33.89,35.49,Kempinski Summerland Hotel & Resort
443,Osmaniye,TR,37.07,36.25,Osmaniye Dynasty Hotel


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))